In [ ]:
all_labels = ['Atelectasis', 'Consolidation', 'Mass', 'Infiltration', 'No Finding', 'Pneumothorax', 'Effusion', 'Nodule','Pleural_Thickening','Emphysema','Edema','Cardiomegaly','Fibrosis','Pneumonia','Hernia']

In [ ]:
!pip install ijson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.6 MB/s eta 0:00:00


In [25]:
import ijson

def fetch_model_weights_large_json(file_path):
    try:
        # Open the JSON file for streaming
        with open(file_path, 'r') as file:
            # Use ijson to incrementally parse the 'model_weights' array
            parser = ijson.items(file, 'final_weights.item')
            model_weights = [item for item in parser]  # Collect the weights into a list
        return model_weights

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
file_path = '/content/training_info2.json'
weights = fetch_model_weights_large_json(file_path)

if weights is not None:
    print("Model Weights:", weights[:10])


Model Weights: [Decimal('-0.09241695702075958'), Decimal('-0.08089049160480499'), Decimal('-0.1134713888168335'), Decimal('0.02299993298947811'), Decimal('0.022746121510863304'), Decimal('-0.03572862967848778'), Decimal('0.05747848004102707'), Decimal('-0.009774364531040192'), Decimal('0.11136361956596375'), Decimal('-0.09931083023548126')]


In [26]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Sequential
base_mobilenet_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights=None)
model = Sequential()
model.add(base_mobilenet_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(all_labels), activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['binary_accuracy', 'mae'])


In [27]:
import numpy as np

def set_model_weights(model, flattened_weights):
    # Get the original shapes of the model weights
    original_weights = model.get_weights()

    # Initialize index to keep track of where to take the next set of weights from
    idx = 0
    reshaped_weights = []

    # Loop over each layer's original weights and reshape the flattened weights
    for w in original_weights:
        # Get the number of elements in this weight matrix
        num_elements = np.prod(w.shape)

        # Reshape the corresponding flattened weights back to the original shape
        reshaped_weight = np.array(flattened_weights[idx:idx + num_elements]).reshape(w.shape)

        # Append the reshaped weight
        reshaped_weights.append(reshaped_weight)

        # Update the index
        idx += num_elements

    # Set the reshaped weights back into the model
    model.set_weights(reshaped_weights)


In [30]:
set_model_weights(model, weights)

In [31]:
import numpy as np

def get_model_weights(model):
    # Get the weights of the model and flatten them
    flattened_weights = np.concatenate([w.flatten() for w in model.get_weights()])
    return flattened_weights.tolist()


In [32]:
w = get_model_weights(model)

print(w[:10])

[-0.09241695702075958, -0.08089049160480499, -0.1134713888168335, 0.02299993298947811, 0.022746121510863304, -0.03572862967848778, 0.05747848004102707, -0.009774364531040192, 0.11136361956596375, -0.09931083023548126]
